In [0]:
#from colab word2vec updated
import time
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf
import preprocess_util
from collections import Counter
import random


In [0]:
!ln -s ./drive/My\ Drive/HornMorph/
!cp ./drive/My\ Drive/Amh2

In [0]:
RAW_DATA_FLD = "./raw_data"
RAW_DATA_FILE = "amharic.txt"
CLEANED_DATA_SAVE_PATH = "./processed_data/cleaned_amh_text.txt"


DATA_SRC_HIS = RAW_DATA_FLD + "/Amharic_English_History/" + RAW_DATA_FILE
DATA_SRC_NEWS = RAW_DATA_FLD + "/Amharic_English_News/" + RAW_DATA_FILE
DATA_SRC_LAW = RAW_DATA_FLD + "/Amharic_English_Legal/" + RAW_DATA_FILE
DATA_SRC_ETH_BIBLE = RAW_DATA_FLD + "/Amharic_English_Ethiopic_Bible/" + RAW_DATA_FILE
DATA_SRC_JW_BIBLW = RAW_DATA_FLD + "/Amharic_English_JW_Bible/" + RAW_DATA_FILE


In [0]:
with open(DATA_SRC_HIS) as f:
    text_his = f.read()

with open(DATA_SRC_NEWS) as f:
    text_news = f.read()

with open(DATA_SRC_LAW) as f:
    text_law = f.read()

with open(DATA_SRC_ETH_BIBLE) as f:
    text_ebible = f.read()

with open(DATA_SRC_ETH_BIBLE) as f:
    text_jw_bible = f.read()

In [0]:
raw_text = text_his + " " + text_news + " " + text_law + " " + text_ebible + " " + text_jw_bible

In [0]:
cleaned_text = preprocess_util.clean_data(raw_text, save=True, savePath=CLEANED_DATA_SAVE_PATH)

In [0]:
#checkpoin1 cleaned data

In [0]:
import word2vec_helper as helper
import time
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf
import preprocess_util
from collections import Counter
import random


>>>>> This is L3Morpho, version 3.0 <<<<<
>>>>>  and HornMorpho, version 2.5  <<<<<
TensorFlow 1.x selected.


In [0]:
with open(CLEANED_DATA_SAVE_PATH) as f:
    cleaned_text = f.read()

In [0]:
tokenized_text, lemmatized_tokenized_text = helper.tokenize(cleaned_text, 3)

In [0]:
preprocess_util.save_dump("./amh_lemmatized_tokens.p", lemmatized_tokenized_text)
print("lemmatized_token_saved...")
#checkpoint 2

lemmatized_token_saved...


In [0]:
lemmatized_tokenized_text = preprocess_util.load_file("./amh_lemmatized_tokens.p")

In [0]:
vocab_to_int, int_to_vocab = helper.create_lookup_tables(lemmatized_tokenized_text)


In [0]:
int_words = [vocab_to_int[word] for word in lemmatized_tokenized_text]

In [0]:
threshold = 1e-5
word_counts = Counter(int_words)
total_count = len(int_words)
freqs = {word: count/total_count for word, count in word_counts.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
train_words = [word for word in int_words if random.random() < (1 - p_drop[word])]

In [0]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = set(words[start:idx] + words[idx+1:stop+1])
    
    return list(target_words)

In [0]:
def get_batches(words, batch_size, window_size=5):
    n_batches = len(words)//batch_size
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y

In [0]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, [None], name='inputs')
    labels = tf.placeholder(tf.int32, [None, None], name='labels')

In [0]:
n_vocab = len(int_to_vocab)
n_embedding =  256
with train_graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_vocab, n_embedding), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs)

In [0]:
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal((n_vocab, n_embedding), stddev=0.1))
    softmax_b = tf.zeros(n_vocab)
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, 
                                      labels, embed,
                                      n_sampled, n_vocab) 
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

In [0]:
with train_graph.as_default():
    ## From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [0]:
!mkdir embedding_checkpoints

mkdir: cannot create directory ‘embedding_checkpoints’: File exists


In [0]:
epochs = 100
batch_size = 2000
window_size = 10

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                ## From Thushan Ganegedara's implementation
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
        save_path = saver.save(sess, "embedding_checkpoints/amh_emebdding.ckpt")
    embed_mat = sess.run(normalized_embedding)

In [0]:
preprocess_util.save_dump("./amh_vocab_mapping.p",(int_to_vocab, vocab_to_int))

In [0]:
with train_graph.as_default():
    saver = tf.train.Saver()
with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('embedding_checkpoints'))
    embed_mat = sess.run(embedding)

INFO:tensorflow:Restoring parameters from embedding_checkpoints/amh_emebdding.ckpt
